In [3]:
import os
import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ShuffleSplit
import random
from string import ascii_letters
import torch
from torch import nn
import torch.nn.functional as F

In [4]:
import ojsim
sim = ojsim.OJSimulator()
X,y = sim.formulized_train
X.shape, y.shape

MemoryError: Unable to allocate 1.90 GiB for an array with shape (17709, 1440, 10) and data type float64

In [ ]:
# fourier on log price
X_pr = X[-8000:,0,:,:]

from joblib import delayed,Parallel, parallel_backend
def inner(i,j):
    window = 20
    filtered = np.zeros(40,dtype=complex)
    raw_fft = np.fft.fft(X_pr[i,:,j])
    filtered[:window] = raw_fft[:window]
    filtered[-window:] = raw_fft[-window:]
    return np.fft.ifft(filtered)

def outer(i):
    return np.array(Parallel(n_jobs=-1)(delayed(inner)(i,j) for j in range(10)),dtype = complex)

result = np.array(Parallel(n_jobs=-1)(delayed(outer)(i) for i in range(8000)),dtype = complex)
input = np.concatenate((result.real, result.imag), axis = 2)
input.shape

In [ ]:
output = y[-8000:,:]

In [5]:
def FastAllFeatureExtract(A):
    d = 28
    A0_pr = A[:, :1440]
    A0_vo = A[:, 1440:] + 1
    feature = A0_pr[:, -1] - A0_pr[:, 0]
    # VO volumn log################
    feature = np.concatenate((feature[:, None], np.log(A0_vo[:, -30+d:])), axis=1)

    # VO moving avg (#sample,91:120)########
    avg_step = 30
    df_vo = pd.DataFrame(A0_vo[:, -59+d:].T)
    ma_30 = lambda x: x.rolling(avg_step).mean()
    df_vo.apply(ma_30).apply(np.log).T.to_numpy()[:, avg_step - 1:]
    feature = np.append(feature, df_vo.apply(ma_30).apply(np.log).T.to_numpy()[:, -30+d:], axis=1)

    # PR rate of change (#sample, 271:300)#######
    df_pr = pd.DataFrame(A0_pr[:, -31:].T)
    pct_chg_fxn = lambda x: x.pct_change()
    # print("PR rate of change", df_pr.apply(pct_chg_fxn).T.to_numpy()[:,-30:])
    feature = np.append(feature, df_pr.apply(pct_chg_fxn).T.to_numpy()[:, -30+d:], axis=1)

    # PR moving avg (#sample,301:330)  #######
    df_pr = pd.DataFrame(A0_pr[:, -59:].T)
    avg_step = 30
    ma_30 = lambda x: x.rolling(avg_step).mean()
    df_pr.apply(ma_30).T.to_numpy()[:, avg_step - 1:]
    # print("PR moving avg", df_pr.apply(ma_30).T.to_numpy()[:,-30:])
    feature = np.append(feature, df_pr.apply(ma_30).T.to_numpy()[:, -30+d:], axis=1)

    # PR binning (#sample, 331:360)#########
    df_pr = pd.DataFrame(A0_pr[:, -30+d:].T)
    n_bins = 10
    bin_fxn = lambda y: pd.qcut(y, q=n_bins, labels=range(1, n_bins + 1))
    binning = df_pr.apply(bin_fxn).T
    # print("PR binning", binning.to_numpy()[:,-30:])
    feature = np.append(feature, binning.to_numpy(), axis=1)

    return feature

In [6]:
# extract features
X_vo = X[-8000:,1,:,:]
X_total = np.concatenate((X_pr,X_vo),axis = 1)
def Feature(i):
    return FastAllFeatureExtract(X_total[:,:,i])
feature_extract = np.array(Parallel(n_jobs=-1)(delayed(Feature)(i) for i in range(10)))
feature_extract = feature_extract.swapaxes(0,1)
feature_extract.shape

(8000, 10, 11)

In [7]:
# Get the input
IInput =np.concatenate((input,feature_extract),axis=2)
IInput.shape, output.shape

((8000, 10, 91), (8000, 10))

In [ ]:
# train the pairwise correlation
from sklearn.ensemble import GradientBoostingRegressor
from joblib import Parallel, delayed
from pickle import dump
def train(i):
    reg = GradientBoostingRegressor().fit(IInput[:,i,:],output[:,i])
    dump(reg, open(f'{i}.FFXGB','wb'))
linear_models = Parallel(n_jobs=-1)(delayed(train)(i) for i in range(10))

In [8]:
from pickle import load
linear_models = []
for i in range(10):
    linear_models.append(load(open(f'{i}.FFXGB','rb')))

In [9]:
OOutput = np.empty((8000,10),dtype=float)
for i in range(8000):
    OOutput[i,:] = np.array([linear_models[j].predict(IInput[i,[j],:]) for j in range(10)]).reshape(-1)

In [10]:
def get_batch(X, Y, batch_size=10000):
    X, Y = X.float(), Y.float()
    n = X.size()[0]
    indices = torch.randint(low=0, high=n, size=(batch_size,))
    return X[indices], Y[indices]

In [11]:
IIIInput = OOutput - X_pr[:,-1,:]
YYput = output - X_pr[:,-1,:]

In [12]:
from tqdm import  tqdm
class Rescale(nn.Module):
    def __init__(self):
        super(Rescale, self).__init__()
        # fc = []
        # for i in range(10):
        #     fc.append(
        #         nn.Linear(1, 1)
        #     )


        # self.fc = nn.ModuleList(fc)
        self.zzw = 10*torch.rand((10,1))#10*torch.FloatTensor(np.ones((10,1)))
        self.zzw.requires_grad_()


    def forward(self, x):
        #print((x[:,0])[:,None].shape)
        out = torch.empty((x.shape),dtype=torch.float)
        for i in range(x.shape[0]):
            for j in range(10):
            #print(x[i,:].shape)
                out[i,j] = x[i,j] * self.zzw[j] *self.zzw[j]
                # out[i,j] = self.fc[j]((x[i,j])[None,None])
        return out
        #return torch.FloatTensor([self.fc[i]((x[:,i])[:,None]) for i in range(10)])



final_model = Rescale()               # Moving the model to GPU if available
criterion = nn.MSELoss()                         # One possible loss criterion
learning_rate = 100
optimizer = torch.optim.SGD([final_model.zzw],lr=learning_rate)#torch.optim.SGD
# optimizer = torch.optim.SGD(final_model.parameters(), lr=learning_rate, momentum=0.9)

training_steps = 1000
for i in tqdm(range(training_steps)):
    minibatch_x, minibatch_y = get_batch(torch.FloatTensor(IIIInput), torch.FloatTensor(YYput))
    output = final_model(minibatch_x)
    stacked = torch.stack([output, minibatch_y]) #2 x b x 10
    stacked = stacked.transpose(1,2).transpose(0,1) # 10 x 2 x b
    corrs = torch.stack([torch.corrcoef(stacked[i])[0,1] for i in range(10)]) # 10 x 1
    loss = -torch.mean(corrs)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(i,loss)

  0%|          | 1/1000 [00:13<3:45:44, 13.56s/it]

0 tensor(-0.5205, grad_fn=<NegBackward0>)


  0%|          | 2/1000 [00:33<4:51:23, 17.52s/it]

1 tensor(-0.5287, grad_fn=<NegBackward0>)


  0%|          | 3/1000 [00:54<5:16:57, 19.07s/it]

...

7 tensor(-0.5271, grad_fn=<NegBackward0>)


  1%|          | 9/1000 [03:28<7:11:10, 26.11s/it]

8 tensor(-0.4989, grad_fn=<NegBackward0>)


  1%|          | 9/1000 [03:32<6:30:40, 23.65s/it]


KeyboardInterrupt: 

In [13]:
final_model.zzw

tensor([[1.5849],
        [6.2551],
        [4.9892],
        [1.2924],
        [0.9898],
        [9.3185],
        [4.2212],
        [2.9166],
        [2.5077],
        [4.1427]], requires_grad=True)

In [75]:
# save model
torch.save(final_model.state_dict(), "final_scale01.mdl")
# load model
# final_model = Rescale()
# final_model.load_state_dict(torch.load("final_scale.mdl"))
# final_model.eval()

In [14]:
def get_r_hat(A,B):
    A = A.values.T
    B = B.values.T
    features = FastAllFeatureExtract(np.concatenate((A,B), axis = 1))
    window = 20
    def getNpredict(i):
        filtered = np.zeros(40,dtype=complex)
        raw_fft = np.fft.fft(A[i,:])
        filtered[:window] = raw_fft[:window]
        filtered[-window:] = raw_fft[-window:]
        hi = np.append(filtered.real,filtered.imag)[None,:]
        hi = np.concatenate((hi,features[[i],:]),axis = 1)
        return linear_models[i].predict(hi)[0]- A[i,-1]
    before_scale =  torch.FloatTensor([getNpredict(i) for i in range(10)] - A[:,-1])
    after_scale =  final_model(before_scale[None,:])
    #return np.array([getNpredict(i) for i in range(10)]) * np.array([3,8,4,8,4,1,8,4,10,9])#
    return after_scale.detach().numpy()

import ojsim
sim = ojsim.OJSimulator()
sim.submit(get_r_hat)

100%|██████████| 8496/8496 [04:19<00:00, 32.79it/s]

Total time used: 259.117s
Pairwise correlation:
	asset 0 = 0.00510
	asset 1 = 0.04444
	asset 2 = 0.04673
	asset 3 = 0.02996
	asset 4 = 0.05310
	asset 5 = 0.02349
	asset 6 = 0.03267
	asset 7 = 0.02625
	asset 8 = 0.04223
	asset 9 = 0.03679
	mean correlation = 0.03407
Overall correlation: -0.01010
Fail to outperform Ziwei's method, whose pairwise average
and overall correlations are (0.02840, 0.01536)


# without NN
Total time used: 210.160s
Pairwise correlation:
	asset 0 = 0.01060
	asset 1 = 0.04222
	asset 2 = 0.04612
	asset 3 = 0.02012
	asset 4 = 0.04990
	asset 5 = 0.02483
	asset 6 = 0.03063
	asset 7 = 0.02822
	asset 8 = 0.02789
	asset 9 = 0.04412
	mean correlation = 0.03247
Overall correlation: -0.00005
===============================
Fail to outperform Ziwei's method, whose pairwise average
and overall correlations are (0.02840, 0.01536)
===============================

# NN with bias
Total time used: 214.427s
Pairwise correlation:
	asset 0 = 0.00510
	asset 1 = 0.04444
	asset 2 = 0.04673
	asset 3 = 0.02996
	asset 4 = -0.05310
	asset 5 = -0.02349
	asset 6 = 0.03267
	asset 7 = -0.02625
	asset 8 = 0.04223
	asset 9 = -0.03679
	mean correlation = 0.00615
Overall correlation: -0.01309
===============================
Fail to outperform Ziwei's method, whose pairwise average
and overall correlations are (0.02840, 0.01536)
===============================

In [ ]:
import ojsim
import main
sim = ojsim.OJSimulator()
sim.submit(main.get_r_hat)

C:\VApps\Anaconda3\envs\stats601\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 20%|██        | 1736/8496 [00:47<03:06, 36.16it/s]